In [ ]:
!pip install -q -U google-generativeai

In [ ]:
from google.colab import userdata
import google.generativeai as genai
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

: 

In [ ]:
# Plate Number Recognition from the frames of a speicific local video

from google.colab import userdata
import google.generativeai as genai
import httpx
import base64
from IPython.display import Markdown, display
import os
import cv2

# Install necessary packages
!pip install -q -U google-generativeai
!pip install opencv-python-headless

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

# Choose a Gemini model
model = genai.GenerativeModel(model_name="gemini-1.5-flash")

def process_image_input(image_input):
    """Converts image from URL or file path to base64 encoding."""
    if isinstance(image_input, str) and image_input.startswith("http"):
        response = httpx.get(image_input)
        if response.status_code == 200:
            return base64.b64encode(response.content).decode("utf-8")
        else:
            raise ValueError("Failed to retrieve image from URL")
    elif os.path.isfile(image_input):
        with open(image_input, "rb") as file:
            return base64.b64encode(file.read()).decode("utf-8")
    else:
        raise ValueError("Invalid input: must be either a file path or a valid image URL.")

def analyze_video(video_path):
    """Extract frames from a video and analyze license plates."""
    try:
        # Open the video and extract frames
        vidcap = cv2.VideoCapture(video_path)
        success, image = vidcap.read()
        count = 0
        results = []

        while success:
            frame_filename = f"frame{count}.jpg"
            cv2.imwrite(frame_filename, image)  # Save frame as JPEG file

            image_data = process_image_input(frame_filename)

            prompt = """
            Please analyze the image of a vehicle's license plate and output the following details in JSON format:
            {
                'plate': 'B 1234 ABC',
                'vehicle': 'car',
                'type': 'sedan',
                'color': 'red',
                'expiry_date': '07 - 17',
                'gate_open_time': '2024-12-02 18:15:01',
                'gate_closed_time': 'N/A'
            }. Only return the JSON object, and use 'N/A' for missing or unknown information.
            """
            # Send the image data to the model
            response = model.generate_content(
                [
                    {
                        "mime_type": "image/jpeg",
                        "data": image_data,
                    },
                    prompt,
                ]
            )

            try:
                # Directly display the response as markdown instead of decoding it
                print(f"Frame {count + 1}:")
                display(Markdown(response.text))
            except Exception as e:
                print(f"Error displaying response: {e}")

            success, image = vidcap.read()  # Read the next frame
            count += 1

            # Process only the first 5 frames or adjust as needed
            if count >= 5:
                break

        # Clean up the frame files
        for i in range(count):
            os.remove(f"frame{i}.jpg")

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

video_path = "media/videos/plate_number_video.mp4" # Put here the path to your video

# Analyze the video
analyze_video(video_path)


```json
{
    'plate': 'B 1308 RFK',
    'vehicle': 'car',
    'type': 'SUV',
    'color': 'black',
    'expiry_date': '09-22',
    'gate_open_time': 'N/A',
    'gate_closed_time': 'N/A'
}
```